# Clasificación rápida en ALeRCE utilizando selección de características y algoritmos genéticos.

Integrantes: Joaquín Cubelli - Tomás de la Sotta

Profesor de Cátedra: Pablo Estévez<br>
Profesor Auxiliar: Ignacio Reyes<br>
Ayudantes: Francisca Cona, Jhon Intriago, Pablo Montero, Óscar Pimentel, Esteban Reyes, Mauricio Romero

El broker astronómico ALeRCE clasifica objetos en tiempo real todas las noches. Para ello utiliza dos modelos, un clasificador basado en estampillas para detección rápida de transientes y un clasificador basado en curvas de luz que utiliza una taxonomía detallada. El clasificador de curvas de luz, descrito en Sánchez-Sáez et al. (2020), consiste en la extracción de múltiples características a partir de las curvas de luz (y otras fuentes de datos) y un modelo basado en Random Forests.

El objetivo de este proyecto es reducir el número de características utilizadas por el clasificador de curvas de luz de ALeRCE, explorando el compromiso entre el desempeño en clasificación y el costo computacional asociado.

La primera etapa del proyecto consiste en **evaluar la estrategia greedy para seleccionar las características más relevantes al momento de clasificar**. En esta estrategia se explorará una característica a la vez, buscando cuál de ellas ayuda más al desempeño del modelo cuando es agregada al conjunto de características utilizadas. También se deberá probar comenzando con todas las características y removiendo una característica a la vez. 

Notar que en esta etapa no se toma en cuenta explícitamente el costo de computar cada característica al momento de efectuar la selección.

Ref: Sánchez-Sáez, P., et al. "Alert Classification for the ALeRCE Broker System: The Light Curve Classifier." arXiv preprint arXiv:2008.03311 (2020).
Ref: Huang, Cheng-Lung, and Chieh-Jen Wang. "A GA-based feature selection and parameters optimization for support vector machines." Expert Systems with applications 31.2 (2006): 231-240. Nota: Tomar como referencia, no como pauta a la cual seguir al pie de la letra. 

Pasos a seguir:
- Dividir los datos en entrenamiento y test (70 %, 30 %), con particionamiento estratificado (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) y normalizar features. Se recomienda QuantileTransformer de scikit-learn. Si el feature no está disponible puede usar un valor -1.0 por ejemplo. Recuerden descartar features baneadas.
- Entrenar un Random Forest balanceado (https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.ensemble.BalancedRandomForestClassifier.html).
- Reportar métricas de desempeño (Accuracy, F1-score).
- Identificar si existe algún pack de features irrelevantes (i.e. que al sacarlo no cambia el desempeño).
- Implementar estrategia greedy (debe estar corriendo para meeting 2, aunque tal vez no tengan listos los resultados o ambos métodos greedy)

## Te voy a pedir que sigas instrucciones del final Dela.

In [1]:
# Correr una vez.

!pip install --upgrade pip
!pip install pyarrow

Requirement already up-to-date: pip in /Users/joaquincubelli/opt/anaconda3/lib/python3.7/site-packages (20.2.4)


In [2]:
# Imports:

# Procesamiento de datos.
from pathlib import Path

import sys
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

# Tensorflow y asociados.

import tensorflow as tf

## Datos para el Proyecto:

Detecciones: https://droppy.alerce.online/$/LLaeQ

No detecciones: https://droppy.alerce.online/$/PrIbP

Features: https://droppy.alerce.online/$/3x3qh

Etiquetas: https://droppy.alerce.online/$/XHsJO

Abrir con pandas. Algunos archivos tienen formato pickle (o parquet (?) https://arrow.apache.org/docs/python/parquet.html), otros CSV. 
Hay features que no deben usar, como por ejemplo el brillo promedio, ya que no queremos sesgar el análisis según qué tan cerca está el objeto.
Los _1 y _2 de los features indican a qué banda pertenece (banda g, banda r).

In [18]:
# Los features que no deben usar son:

banned_features = [
   'mean_mag_1',
   'mean_mag_2',
   'min_mag_1',
   'min_mag_2',
   'Mean_1',
   'Mean_2',
   'n_det_1',
   'n_det_2',
   'n_pos_1',
   'n_pos_2',
   'n_neg_1',
   'n_neg_2',
   'first_mag_1',
   'first_mag_2',
   'MHPS_non_zero_1',
   'MHPS_non_zero_2',
   'MHPS_PN_flag_1',
   'MHPS_PN_flag_2',
   #'W1', 'W2', 'W3', 'W4',
   'iqr_1',
   'iqr_2',
   'delta_mjd_fid_1',
   'delta_mjd_fid_2',
   'last_mjd_before_fid_1',
   'last_mjd_before_fid_2'#,
   #'g-r_ml',
   #'MHAOV_Period_1', 'MHAOV_Period_2'
]

# Datos comentados no se observaban en detecciones ni en no_detecciones

In [20]:
len(banned_features)

24

En el archivo con las etiquetas hay varias clases. Para el análisis agruparemos algunas clases según el siguiente diccionario (etiqueta original: nueva etiqueta)

{
 "class_dictionary": {
   "AGN": "AGN",
   "Blazar": "Blazar",
   "CV/Nova": "CV/Nova",
   "SNIa": "SNIa",
   "SNIbc": "SNIbc",
   "SNII": "SNII",
   "SNIIn": "SNII",
   "SLSN": "SLSN",
   "EA": "E",
   "EB/EW": "E",
   "DSCT": "DSCT",
   "RRL": "RRL",
   "Ceph": "CEP",
   "LPV": "LPV",
   "Periodic-Other": "Periodic-Other",
   "QSO": "QSO",
   "YSO": "YSO",
   "RSCVn": "Periodic-Other"
 }
}

In [4]:
# Directorios de datasets

etiquetas_dir = "/Users/joaquincubelli/Desktop/Inteligencia Computacional/ALeRCE_data/Etiquetas/dfcrossmatches_prioritized_v7.0.1.csv" # csv
features_dir = "/Users/joaquincubelli/Desktop/Inteligencia Computacional/ALeRCE_data/storage/ztf_workspace/historic_data_20200916/features_20200916.parquet" # parquet
# Los siguientes directorios tienen datasets separados en múltiples archivos.
detections_dir = Path("/Users/joaquincubelli/Desktop/Inteligencia Computacional/ALeRCE_data/detections_with_xmatch/") # parquet
no_detections_dir = Path("/Users/joaquincubelli/Desktop/Inteligencia Computacional/ALeRCE_data/notdet_with_xmatch/") # parquet

### Lectura Etiquetas:

Por lo visto, en este dataset estan los nombres de cada objeto, junto con la clase respectiva, la posición, período, e información adicional de fuente.

Para este problema, no parece ser muy importante el archivo, salvo por la relación entre el nombre y el tipo de clase.

In [7]:
etiquetas = pd.read_csv(etiquetas_dir)
etiquetas

,oid,classALeRCE,ra,dec,period,source,id_source,class_source,separation_arcsec
0,ZTF19abegncu,AGN,357.296363,-8.941186,NaN,Oh2015,5.8772718059579e+17,AGN_galaxy_dominated,0.243081
1,ZTF18acejdhu,AGN,11.958444,-10.496615,NaN,Oh2015,5.877272256953059e+17,AGN_galaxy_dominated,0.301126
2,ZTF18acdzene,AGN,17.478620,-10.110250,NaN,Oh2015,5.877271789938934e+17,AGN_galaxy_dominated,0.158643
3,ZTF18abwzuzw,AGN,25.038255,-10.352430,NaN,Oh2015,5.877272294485526e+17,AGN_galaxy_dominated,0.485322
4,ZTF19abmposz,AGN,26.670031,-8.354787,NaN,Oh2015,5.877271806085038e+17,AGN_galaxy_dominated,0.213990
...,...,...,...,...,...,...,...,...,...
173874,ZTF18abikbvc,LPV,286.720222,24.733440,NaN,GAIADR2VS,4.533629080944307e+18,MIRA_SR,0.299487
173875,ZTF19aamtoyb,LPV,99.257590,-15.567786,NaN,GAIADR2VS,2.9501070744892713e+18,MIRA_SR,0.299597
173876,ZTF19abeesgk,LPV,277.499222,-27.002742,NaN,GAIADR2VS,4.0518162825206625e+18,MIRA_SR,0.299766
173877,ZTF18abnubuy,LPV,292.053754,38.418277,NaN,GAIADR2VS,2.0526959231045647e+18,MIRA_SR,0.299777


### Lectura Features:

Por lo visto, este es el archivo a revisar, puesto a que tiene los nombres de cada objeto, con cada feature característica.
Sin embargo, en este archivo no aparece la clase de cada objeto, por lo que es necesario relacionar ambas databases para clasificación. 

Recordar que en las instrucciones nos muestran cómo manejar los NaN:
*Si el feature no está disponible puede usar un valor -1.0 por ejemplo. Recuerden descartar features baneadas.*

In [5]:
features_file = pq.read_table(features_dir)

In [6]:
features = features_file.to_pandas()
features

,index,Amplitude_1,Amplitude_2,AndersonDarling_1,AndersonDarling_2,Autocor_length_1,Autocor_length_2,Beyond1Std_1,Beyond1Std_2,Con_1,...,n_non_det_after_fid_1,n_non_det_after_fid_2,n_non_det_before_fid_1,n_non_det_before_fid_2,n_pos_1,n_pos_2,positive_fraction_1,positive_fraction_2,rb,sgscore1
0,ZTF17aaaafan,0.094794,0.269390,0.500701,0.997029,2.0,1.0,0.285714,0.571429,0.0,...,179.0,151.0,6.0,11.0,0.0,3.0,0.000000,0.428571,0.644286,0.984375
1,ZTF17aaaafbs,0.399396,0.368885,1.000000,1.000000,1.0,1.0,0.294118,0.308824,0.0,...,134.0,117.0,7.0,1.0,36.0,38.0,0.705882,0.558824,0.764286,0.986917
2,ZTF17aaabdlz,0.151955,0.143481,1.000000,1.000000,1.0,1.0,0.214286,0.409091,0.0,...,86.0,88.0,1.0,0.0,11.0,13.0,0.785714,0.590909,0.720714,0.977125
3,ZTF17aaabelc,0.247272,0.114171,1.000000,0.481141,1.0,2.0,0.272727,0.400000,0.0,...,76.0,103.0,3.0,14.0,16.0,0.0,0.727273,0.000000,0.851190,1.000000
4,ZTF17aaabgdt,0.164834,NaN,0.762395,NaN,1.0,NaN,0.285714,NaN,0.0,...,10.0,NaN,0.0,NaN,3.0,NaN,0.428571,NaN,0.874286,0.942000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3076,ZTF20abmyskb,NaN,0.087336,NaN,0.163038,NaN,1.0,NaN,0.285714,NaN,...,NaN,8.0,NaN,2.0,NaN,0.0,NaN,0.000000,0.894286,0.990833
3077,ZTF20abmzwlk,NaN,0.048049,NaN,0.655232,NaN,1.0,NaN,0.222222,NaN,...,NaN,23.0,NaN,9.0,NaN,9.0,NaN,1.000000,0.902857,0.500000
3078,ZTF20abmzxbb,0.095092,0.083899,0.921819,0.160114,1.0,1.0,0.142857,0.454545,0.0,...,14.0,7.0,10.0,3.0,7.0,11.0,1.000000,1.000000,0.853571,0.995521
3079,ZTF20abnejau,NaN,0.025581,NaN,0.387126,NaN,1.0,NaN,0.250000,NaN,...,NaN,80.0,NaN,25.0,NaN,0.0,NaN,0.000000,0.670000,0.983476


In [19]:
#features[banned_features] # El archivo de features parece tener en parte las banned features, salvo por las banned features comentadas al inicio.

,mean_mag_1,mean_mag_2,min_mag_1,min_mag_2,Mean_1,Mean_2,n_det_1,n_det_2,n_pos_1,n_pos_2,...,MHPS_non_zero_1,MHPS_non_zero_2,MHPS_PN_flag_1,MHPS_PN_flag_2,iqr_1,iqr_2,delta_mjd_fid_1,delta_mjd_fid_2,last_mjd_before_fid_1,last_mjd_before_fid_2
0,19.139615,17.600847,19.049640,17.309369,19.139615,17.600847,7.0,7.0,0.0,3.0,...,0.0,0.0,1.0,1.0,0.124453,0.509548,734.934768,437.763426,58292.425799,58319.353669
1,17.954389,17.046977,17.694538,16.745190,17.954389,17.046977,51.0,68.0,36.0,38.0,...,37.0,55.0,0.0,0.0,0.619578,0.504314,784.873947,781.828299,58301.324699,58277.400741
2,16.463794,15.578004,16.393184,15.452600,16.463794,15.578004,14.0,22.0,11.0,13.0,...,0.0,8.0,1.0,0.0,0.027857,0.245289,507.727106,757.946505,58376.486539,NaN
3,17.379541,16.836312,17.199746,16.725138,17.379541,16.836312,22.0,10.0,16.0,0.0,...,9.0,0.0,0.0,1.0,0.319271,0.098441,712.041724,633.131713,58349.450926,58473.351979
4,15.971543,NaN,15.819786,NaN,15.971543,NaN,7.0,NaN,3.0,NaN,...,0.0,NaN,1.0,NaN,0.226245,NaN,425.743426,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3076,NaN,19.905807,NaN,19.810410,NaN,19.905807,NaN,7.0,NaN,0.0,...,NaN,5.0,NaN,1.0,NaN,0.064704,NaN,36.070116,NaN,59038.377836
3077,NaN,18.708162,NaN,18.650655,NaN,18.708162,NaN,9.0,NaN,9.0,...,NaN,8.0,NaN,1.0,NaN,0.059477,NaN,49.929005,NaN,59046.441910
3078,19.601279,19.193073,19.472894,19.113551,19.601279,19.193073,7.0,11.0,7.0,11.0,...,6.0,11.0,0.0,1.0,0.047558,0.058404,35.931111,37.953657,59048.486632,59040.381192
3079,NaN,19.017872,NaN,18.988753,NaN,19.017872,NaN,8.0,NaN,0.0,...,NaN,8.0,NaN,1.0,NaN,0.014806,NaN,40.009838,NaN,59050.465752


Joaquin - No se si sea necesario usar los otros archivos siguientes, estos tienen ciertos parámetros, pero no tienen ni siquiera los banned_features.

### Lectura Detections:

Es más complicado que con los métodos anteriores, encontré esto en stackoverflow:

https://stackoverflow.com/questions/51696655/read-multiple-parquet-files-in-a-folder-and-write-to-single-csv-file-using-pytho

In [8]:
#detections = pd.concat(
#    pd.read_parquet(parquet_file)
#    for parquet_file in detections_dir.glob('*.parquet')
#)
# La lectura de los datos toma unos buenos minutos.

In [9]:
#detections

,fid,isdiffpos,sigmapsf,field,fwhm,dec,magpsf,rcid,ra,sky,...,sgscore3,neargaia,sgscore1,mjd,corrected,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ext,dubious,has_stamp
objectId,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaagyq,1,-1.0,0.081519,743,4.03000,48.965471,18.716232,40,71.640358,-0.652527,...,0.500000,0.297090,1.000000,58426.382870,True,17.668462,0.024305,0.031056,False,True
ZTF17aaaagyq,1,1.0,0.104178,743,3.02000,48.965370,18.252100,40,71.640256,1.662210,...,NaN,NaN,NaN,58693.443437,True,16.934974,0.029365,0.030969,False,True
ZTF17aaaagyq,1,1.0,0.117649,743,2.56000,48.965336,18.525101,40,71.640474,0.695077,...,NaN,NaN,NaN,58567.160382,True,17.009211,0.027151,0.029123,False,True
ZTF17aaaagyq,1,-1.0,0.114034,743,1.85000,48.965447,18.472069,40,71.640350,-0.467757,...,0.500000,0.216508,1.000000,58874.193877,True,17.778127,0.056252,0.060181,False,True
ZTF17aaaagyq,1,1.0,0.193104,743,1.88000,48.965367,19.101601,40,71.640397,-0.143099,...,NaN,NaN,NaN,58488.219907,True,17.125994,0.029019,0.031300,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF20abixgjy,2,1.0,0.201382,444,2.34000,-1.203402,19.820194,37,341.075385,-0.181441,...,0.994375,0.182846,0.985333,59069.402095,True,18.096667,0.038602,0.041173,False,True
ZTF20abixgjy,2,1.0,0.229447,444,1.50260,-1.203489,20.254200,37,341.075416,-0.006699,...,NaN,NaN,NaN,59030.463576,True,18.172390,0.030028,0.033725,False,True
ZTF20abixgjy,2,1.0,0.203909,444,1.04073,-1.203398,19.913500,37,341.075416,0.556812,...,NaN,NaN,NaN,59042.460451,True,18.115102,0.036083,0.038911,False,True


In [14]:
#detections[banned_features]

KeyError: "None of [Index(['mean_mag_1', 'mean_mag_2', 'min_mag_1', 'min_mag_2', 'Mean_1',\n       'Mean_2', 'n_det_1', 'n_det_2', 'n_pos_1', 'n_pos_2', 'n_neg_1',\n       'n_neg_2', 'first_mag_1', 'first_mag_2', 'MHPS_non_zero_1',\n       'MHPS_non_zero_2', 'MHPS_PN_flag_1', 'MHPS_PN_flag_2', 'W1', 'W2', 'W3',\n       'W4', 'iqr_1', 'iqr_2', 'delta_mjd_fid_1', 'delta_mjd_fid_2',\n       'last_mjd_before_fid_1', 'last_mjd_before_fid_2', 'g-r_ml',\n       'MHAOV_Period_1', 'MHAOV_Period_2'],\n      dtype='object')] are in the [columns]"

### Lectura no Detections:

Igualmente que en el caso anterior, este dataset está dividido en multiples archivos, pero a diferencia del anterior, cada archivo está en formato .snappy.parquet.

Joaquín - No logro abrir esto. 

In [10]:
#no_detections = pd.concat(
#    pd.read_parquet(parquet_file)
#    for parquet_file in no_detections_dir.glob('*.snappy.parquet')
#)

ValueError: No objects to concatenate

Joaquín - Ahora que importé los datos, y que sabemos que solo tenemos que usar el dataset de "Features", debemos eliminar las features baneadas, y reemplazar todos los NaN por -1.

In [25]:
features = features.fillna(-1)
features = features.drop(banned_features, axis=1)

In [26]:
features.head()

,index,Amplitude_1,Amplitude_2,AndersonDarling_1,AndersonDarling_2,Autocor_length_1,Autocor_length_2,Beyond1Std_1,Beyond1Std_2,Con_1,...,median_diffmaglim_before_fid_1,median_diffmaglim_before_fid_2,n_non_det_after_fid_1,n_non_det_after_fid_2,n_non_det_before_fid_1,n_non_det_before_fid_2,positive_fraction_1,positive_fraction_2,rb,sgscore1
0,ZTF17aaaafan,0.094794,0.269390,0.500701,0.997029,2.0,1.0,0.285714,0.571429,0.0,...,20.717550,20.547300,179.0,151.0,6.0,11.0,0.000000,0.428571,0.644286,0.984375
1,ZTF17aaaafbs,0.399396,0.368885,1.000000,1.000000,1.0,1.0,0.294118,0.308824,0.0,...,20.374399,20.536400,134.0,117.0,7.0,1.0,0.705882,0.558824,0.764286,0.986917
2,ZTF17aaabdlz,0.151955,0.143481,1.000000,1.000000,1.0,1.0,0.214286,0.409091,0.0,...,20.901699,-1.000000,86.0,88.0,1.0,0.0,0.785714,0.590909,0.720714,0.977125
3,ZTF17aaabelc,0.247272,0.114171,1.000000,0.481141,1.0,2.0,0.272727,0.400000,0.0,...,20.994600,19.735451,76.0,103.0,3.0,14.0,0.727273,0.000000,0.851190,1.000000
4,ZTF17aaabgdt,0.164834,-1.000000,0.762395,-1.000000,1.0,-1.0,0.285714,-1.000000,0.0,...,-1.000000,-1.000000,10.0,-1.0,0.0,-1.0,0.428571,-1.000000,0.874286,0.942000


Dela, **Falta relacionar el índice con su respectiva clase, y agregarla al DataFrame de features, para luego separar el dataset en el training y testing set.** Si alcanzas, además crea el Random Forest, y entrénalo.